In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# TODO: test is reinit ClaudeSDKClient needed after change ClientOptions

In [ ]:
import asyncio
import json
from claude_agent_sdk import Message, UserMessage, AssistantMessage, SystemMessage, ResultMessage
from claude_agent_sdk import ContentBlock, TextBlock, ThinkingBlock, ToolUseBlock, ToolResultBlock
from claude_agent_sdk import query, ClaudeAgentOptions
from dataclasses import dataclass, asdict

async def main():
    options = ClaudeAgentOptions(
        system_prompt="You are an expert Python developer",
        permission_mode='acceptEdits',
        cwd="/home/ahggg/desktop/github/side-project/aiswitch"
    )

    async for message in query(
        prompt="1+1=?",
        options=options
    ):
        print(f"type: {type(message)}\nto_dict: {asdict(message)}\n\n")


await main()

type: <class 'claude_agent_sdk.types.SystemMessage'>
to_dict: {'subtype': 'init', 'data': {'type': 'system', 'subtype': 'init', 'cwd': '/home/ahggg/desktop/github/side-project/aiswitch', 'session_id': '5827bcdd-ebf3-4a02-add8-8b5ba395281b', 'tools': ['Task', 'Bash', 'Glob', 'Grep', 'ExitPlanMode', 'Read', 'Edit', 'Write', 'NotebookEdit', 'WebFetch', 'TodoWrite', 'WebSearch', 'BashOutput', 'KillShell', 'SlashCommand'], 'mcp_servers': [], 'model': 'claude-sonnet-4-5-20250929', 'permissionMode': 'acceptEdits', 'slash_commands': ['compact', 'context', 'cost', 'init', 'output-style:new', 'pr-comments', 'release-notes', 'todos', 'review', 'security-review'], 'apiKeySource': 'none', 'output_style': 'default', 'agents': ['general-purpose', 'statusline-setup', 'output-style-setup'], 'uuid': '9c25f937-e7d9-45b1-92db-4055f87fb491'}}


type: <class 'claude_agent_sdk.types.AssistantMessage'>
to_dict: {'content': [{'text': '1 + 1 = 2'}], 'model': 'claude-sonnet-4-5-20250929', 'parent_tool_use_id': N

### monitor progress

In [13]:
from claude_agent_sdk import (
    ClaudeSDKClient,
    ClaudeAgentOptions,
    AssistantMessage,
    ToolUseBlock,
    ToolResultBlock,
    TextBlock
)
import asyncio

async def monitor_progress():
    options = ClaudeAgentOptions(
        allowed_tools=["Write", "Bash"],
        permission_mode="acceptEdits",
        cwd="/home/ahggg/desktop/github/side-project/aiswitch"
    )
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query(
            "Create 5 Python files with different sorting algorithms"
        )
        
        # Monitor progress in real-time
        files_created = []
        async for message in client.receive_messages():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, ToolUseBlock):
                        if block.name == "Write":
                            file_path = block.input.get("file_path", "")
                            print(f"🔨 Creating: {file_path}")
                    elif isinstance(block, ToolResultBlock):
                        print(f"✅ Completed tool execution")
                    elif isinstance(block, TextBlock):
                        print(f"💭 Claude says: {block.text[:100]}...")
            
            # Check if we've received the final result
            if hasattr(message, 'subtype') and message.subtype in ['success', 'error']:
                print(f"\n🎯 Task completed!")
                break

await monitor_progress()

💭 Claude says: I'll create 5 Python files with different sorting algorithms....
🔨 Creating: /home/ahggg/desktop/github/side-project/aiswitch/bubble_sort.py
🔨 Creating: /home/ahggg/desktop/github/side-project/aiswitch/quick_sort.py
🔨 Creating: /home/ahggg/desktop/github/side-project/aiswitch/merge_sort.py
🔨 Creating: /home/ahggg/desktop/github/side-project/aiswitch/insertion_sort.py
🔨 Creating: /home/ahggg/desktop/github/side-project/aiswitch/selection_sort.py
💭 Claude says: Created 5 Python files with different sorting algorithms:

1. **bubble_sort.py** - Bubble Sort (O(n²...

🎯 Task completed!


### chat_history test

In [14]:
from claude_agent_sdk import ClaudeSDKClient, ClaudeAgentOptions, AssistantMessage, TextBlock
import asyncio
import base64

async def streaming_analysis():
    async def message_generator():
        # First message
        yield {
            "type": "user",
            "message": {
                "role": "user",
                "content": "1+1=?"
            }
        }

        # Wait for conditions
        await asyncio.sleep(2)

        yield {
            "type": "user",
            "message": {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "刚刚我问了你什么?"
                    }
                ]
            }
        }

    # Use ClaudeSDKClient for streaming input
    options = ClaudeAgentOptions(
        max_turns=10,
        allowed_tools=["Read", "Grep"]
    )

    async with ClaudeSDKClient(options) as client:
        # Send streaming input
        await client.query(message_generator())

        # Process responses
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(block.text)

await streaming_analysis()

2


### monitoring todo

In [ ]:
from claude_agent_sdk import query

async for message in query(
    prompt="Optimize my React app performance and track progress with todos",
    options={"max_turns": 15}
):
    # Todo updates are reflected in the message stream
    if message.get("type") == "tool_use" and message.get("name") == "TodoWrite":
        todos = message["input"]["todos"]
        
        print("Todo Status Update:")
        for i, todo in enumerate(todos):
            status = "✅" if todo["status"] == "completed" else \
                    "🔧" if todo["status"] == "in_progress" else "❌"
            print(f"{i + 1}. {status} {todo['content']}")
